# HTML Confusion Matrix and Performance Report Demo
created by Reynaldo Vazquez <br>
April 2021

### Demonstrates the use of the functions in the `html_metrics.py` script.

Why html? I was inspired to create these simply by aesthetics. For some time, I have been using “heated map” confusion matrices as an excuse to add color into otherwise duller notebooks and reports. However, options such as matplotlib and seaborn tended to blurry the text in the plots. This was particularly true when the plots are saved as jpg or png images and then (often automatically) resized to fit different types of displays. 

My way around this was to create these functions, where text is displayed directly into the “graphics” and never converted to image. These can then be displayed in Jupyter notebooks, webpages, or even Microsoft documents. 

Below are examples of how to use the functions in a Jupyter notebook and other html documents. To use in a Microsoft (e.g. Word) document, provide a file name in the `confusion_matrix_html()` function e.g. `html_file = "my_file_name.html"` then drag the created file onto an opened Microsoft document. 




In [1]:
import numpy as np
import pandas as pd
import random, string
from sklearn.metrics import accuracy_score, confusion_matrix
from IPython.display import display, HTML

!wget -q https://raw.githubusercontent.com/reyvaz/utils-aux/master/html_metrics.py
from html_metrics import *

In [2]:
#@markdown Data Simulation Function
def sim_data(n_classes = 2, n_obs = 1000, sim_accuracy = 0.90):
    '''
    Creates simulated ground-truth and prediction data
    Args:
        n_classes: (int) the number of classes to be simulated
        n_obs: (int) the number of observations to be simulated
        sim_accuracy: (float) the approximated simulated accuracy 
    Returns:
        a tuple containing numpy arrays of simulated ground_truth and predictions 
        and a list of class names. 
    '''
    ground_truth = np.random.randint(n_classes, size=n_obs)
    classes = np.arange(0, n_classes, 1)
    class_names = ['Class {}'.format(i) for i in classes]
    impute_factor = n_classes/(n_classes - 1)
    n_impute = round((1-sim_accuracy)*impute_factor*n_obs)
    predictions = ground_truth.copy()
    predictions[:n_impute] = [random.choice(classes) for i in range(n_impute)]
    return ground_truth, predictions, class_names

## Basic Confusion Matrix

Below it shows the html confusion matrix with default argument values. 

By default, `confusion_matrix_html()` displays the confusion matrix but it can 
be suppressed by setting `show_cm=False`. The function always returns the html string.

In [3]:
gt, preds, classes = sim_data()  # simulating data
cm = confusion_matrix(gt, preds) # getting cm from sklearn.metrics.confusion_matrix
print(cm)

[[460  44]
 [ 49 447]]


In [4]:
cm_string = confusion_matrix_html(cm)

In [5]:
print(cm_string)

<style>
.cm {height: 250px; display:table-cell; vertical-align:middle;}
.tg {border-spacing:0;text-align:center; vertical-align:middle}
.tg td {overflow:hidden; padding:8px 6px; word-break:normal;}
.tg th {overflow:hidden; word-break:normal;}
.ver_text {transform: rotate(-90deg);}
.norm {font-size:11.5px}
</style>
<table class="tg">
<thead>
    <tr>
      <th class="tg" colspan="2" rowspan="2"></th>
      <th class="tg" colspan="2">Confusion Matrix<br><span class="norm"></span></th>
    </tr>
    <tr><td class="tg" colspan="2">Predicted</td></tr>
</thead>
<tbody>
    <tr>
	<td class="tg ver_text" rowspan="2">Actual</td>
	<td class="tg">Class 1</td><td class="tg" style="background-color:rgb(14, 57, 75); color:white">460<br><span class="norm">(0.91)</span></td><td class="tg" style="background-color:rgb(195, 203, 207); color:grey">44<br><span class="norm">(0.09)</span></td></tr><tr>
	<td class="tg">Class 2</td><td class="tg" style="background-color:rgb(191, 200, 204); color:grey">49<br><s

The string can then be used with any HTML interpreter, or pasted into an html document. 

In [6]:
display(HTML(cm_string))

## Binary Performance Report
A performance report can be produced as follows

Even though `confusion_matrix_html()` can be used with any number of classes, the tabular part of the report is designed only for binary classes. The following uses the data simulated above, since it simulates binary classes, it should work fine.

In [7]:
_, cm, metrics_strings = performance_metrics(gt, preds)
table_string = metrics_table_html_string(metrics_strings)
display(HTML(html_report_base % (cm_string, table_string)))

## Confusion Matrix with `n>2` Classes

In [8]:
# Create the simulated data with n_classes > 2
gt, preds, classes = sim_data(n_classes=6, n_obs=2000, sim_accuracy=0.92)
cm = confusion_matrix(gt, preds)

Still, `confusion_matrix_html()` only requires the `cm` array

In [9]:
_ = confusion_matrix_html(cm)

### Using custom colors and classes

Color can be specified by the `max_rgb` argument. The argument should be a tuple tuple with the rgb code for the darkest possible cell.

In [10]:
my_classes = list(string.ascii_uppercase[:6])
_ = confusion_matrix_html(cm, classes = my_classes, max_rgb=(204, 0, 90))

## More colors

In [11]:
#@markdown Show Color from RGB
color_palette_base = '''
<style>
.demo {min-width: 120px; padding: 3px 10px; display: inline-block;
    font-size: 14px; color: white; text-align:center; margin: 5px;}
%s
</style>
<div>\n%s\n</div> 
''' 

def show_rgb_colors(rgbs):
    css_str, html_str = '', ''
    for i, c in enumerate(rgbs, 1):
        css_str += '.demo%d {background-color: rgb%s}\n' % (i, c)
        html_str += '\t<span class="demo demo%d">%s</span>\n' % (i, c)
        if i % 4 == 0: html_str += '<br>'

    colors_html = HTML(color_palette_base % (css_str, html_str))
    display(colors_html)
    return None

rgbs = [(153, 0, 61), (0, 102, 0), (153, 115, 0), (0, 43, 128), 
        (0, 85, 128), (115, 38, 115), (179, 71, 0), (128, 128, 0)]
show_rgb_colors(rgbs)

In [12]:
#@markdown Generate Confusion Matrices
num_classes = 6
my_classes = list(string.ascii_uppercase[:num_classes])

html_columns = '''
<style>
.column {float: left; width: %d px; padding: 15px;}
</style>
<div>%s</div>
''' % (200+(15*num_classes), '%s')
cm_divs = ''
for rc in rgbs:
    gt, preds, classes = sim_data(num_classes, 1000, 0.90)
    cm = confusion_matrix(gt, preds)
    html_cm = confusion_matrix_html(cm, classes = my_classes, max_rgb=rc, show_cm=False, supress = False, blank_zeros=False)
    cm_divs += '<div class="column"> {} </div>'.format(html_cm)

display(HTML(html_columns % cm_divs))